# Kafka

O Apache Kafka é uma plataforma de streaming distribuída de software livre que pode ser usada para compilar pipelines e aplicativos de dados de streaming em tempo real. O Kafka também fornece funcionalidade de agente de mensagem semelhante a uma fila de mensagens, onde você pode publicar e assinar os fluxos de dados nomeados.

A API de produtor do Kafka permite que os aplicativos enviam fluxos de dados para o cluster Kafka. A API de consumidor do Kafka permite que os aplicativos leiam fluxos de dados do cluster.

Neste tutorial, você irá aprender como criar produtores e consumidores para clusters Kafka utilizando Python. Iremos instalar a biblioteca `confluent-kafka` que irá nos permitir utilizar a API do Kafka em Python.

In [ ]:
!pip install confluent-kafka

# Produtor

O produtor se comunica com os hosts de agente de Kafka (nós de trabalho) e envia dados a um tópico Kafka.

No código abaixo importamos a classe `Producer` para enviar mensagens para o Kafka.

In [ ]:
from confluent_kafka import Producer

Iremos produzir mensagens *fake* para o tópico do Kafka. Por isso, iremos utilizar a biblioteca faker do Python que gera nomes, endereços, cidades, etc... fakes para podermos construir uma mensagem *fake*.

In [ ]:
!pip install faker

Abaixo as bibliotecas são importadas para serem utilizadas no Produtor.

In [ ]:
import json
import random
from faker import Faker

Iniciamos o objeto Faker que será utilizado para criar dados *fake*.

In [ ]:
fake=Faker()

Para criar um produtor de mensagens, devemos informar onde está o cluster do Kafka. Para isto definimos em `bootstrap.servers` o IP e porta do cluster do Kafka. Neste tutorial, o IP é *kafka* e a porta *9092*.

In [ ]:
producer=Producer({'bootstrap.servers':'kafka:9092'})
print('Kafka Producer foi inicializado...')

### Criação de mensagens

Neste tutorial iremos enviar 10 mensagens para o tópico `usuarios` do Kafka. Iremos enviar uma mensagem com os seguintes campos:

- id_usuario
- nome_usuario
- endereco_usuario
- platforma
- data_cadastro

In [ ]:
for i in range(10):
    # constroi o JSON que será enviado como mensagem
    data={
       'id_usuario': fake.random_int(min=20000, max=100000),
       'nome_usuario':fake.name(),
       'endereco_usuario':fake.street_address() + ' | ' + fake.city() + ' | ' + fake.country_code(),
       'platforma': random.choice(['Celular', 'Notebook', 'Tablet']),
       'data_cadastro': str(fake.date_time_this_month())    
       }
    # converte o JSON para string para ser enviado no Kafka
    m=json.dumps(data)
    # publica a mensagem no tópico 'usuarios' com codificação UTF-8
    producer.produce('usuarios', m.encode('utf-8'))

# faz um flush das mensagens
producer.flush()

# Consumidor

O consumidor se comunica com os hosts de broker Kafka (nós de trabalho) e consome as mensagens de um tópico. O primeiro passo é carregar a classe `Consumer` que será utilizada para consumir mensagens da API do Kafka.

In [ ]:
from confluent_kafka import Consumer

O próximo passo é consumir mensagens do Kafka. Para isto, passamos três parâmetros para o `Consumer`:

- `bootstrap.servers`: ip e porta do Kafka. Aqui o IP é *kafka* e a porta *9092*.
- `group.id`: identificador do grupo de consumidores do Kafka. Os consumidores dentro do grupo revesam no consumo de mensagens do tópico. Aqui definimos *consumidor-usuarios* como identificador do grupo.
- `auto.offset.reset`: define o offset de consumo de mensagens, ou seja, a partir de quando iremos ler mensagens do Kafka. Aqui definimos `earliest`, ou seja, iremos consumir todas as mensagens do tópico. Se colocarmos, por exemplo, `latest` iríamos consumir apenas as mensagens que chegarem daqui pra frente.

In [ ]:
consumer=Consumer({'bootstrap.servers':'kafka:9092','group.id':'consumidor-usuarios','auto.offset.reset':'earliest'})
print('Kafka Consumer foi inicializado...')

Para ler mensagens de um tópico, o consumidor deve se inscrever no tópico. No código abaixo o consumidor faz a inscrição no tópico `usuarios`.

In [ ]:
print('Tópicos disponíveis: ', consumer.list_topics().topics)
consumer.subscribe(['usuarios'])

Depois de inscrito no tópico chegou o momento de ler mensagens do tópico. O código abaixo itera no tópico `usuarios` lendo todas as mensagens. Quando não houver mais mensagens para serem lidas, o loop finaliza.

In [ ]:
while True:
    # consome uma mensagem do tópico (espera 1 segundo)
    msg=consumer.poll(1.0) #timeout
    # Se a mensagem for nula é porque não existem mais mensagens.
    # Neste caso, sai do loop
    if msg is None:
        break
    
    # Se tiver erro na mensagem lida, imprime a mensagem e continua o loop.
    if msg.error():
        print('Erro: {}'.format(msg.error()))
        continue
    
    # lê o valor da mensagem
    data=msg.value().decode('utf-8')
    #imprime cada mensagem do tópico
    print(data)

Depois de lida todas as mensagens do tópico, o consumidor fecha a conexão com o Kafka no código abaixo.

In [ ]:
consumer.close()

# Tarefa

Agora chegou sua vez de escrever e ler mensagens de um tópico do Kafka! Nesta tarefa você deve:

- **Produtor**: escrever 20 mensagens no tópico `alunos` do Kafka. Faça um flush ao final da escrita.
- **Consumidor**: ler as 20 mensagens do tópico `alunos` do Kafka. O `group-id` deve ser 'consumidor-alunos'. Feche a conexão com o Kafka ao final da leitura.
- Cada mensagem deve ser construída da seguinte forma:


```code
data={
       'id_aluno': fake.random_int(min=20000, max=100000),
       'nome_aluno':fake.name(),
       'endereco_aluno':fake.street_address() + ' | ' + fake.city() + ' | ' + fake.country_code(),
       'nota_aluno': fake.random_int(min=0, max=100)
       }
```
